# Разбор возможностей Hugging Face с использованием бесплатных моделей
Hugging Face предлагает множество бесплатных предобученных моделей, которые можно использовать для обработки естественного языка (NLP). В данном руководстве посмотрим, как загружать и применять эти модели, используя библиотеку transformers.

Hugging Face предлагает библиотеку transformers для работы с моделями и datasets для работы с открытыми наборами данных.

In [ ]:
# !pip install transformers datasets torch

### 2. Загрузка бесплатной модели для работы с текстом
Для начала загрузим предобученную модель BERT и её токенизатор:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Выбираем предобученную бесплатную модель
model_name = "distilbert-base-uncased"

# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print("Модель и токенизатор успешно загружены!")


Здесь мы используем DistilBERT, который является облегчённой версией BERT, обученной без потери качества.

 ## 3. Токенизация текста
Токенизируем входной текст перед подачей в модель:

In [ ]:
text = "Hugging Face makes NLP easy!"
inputs = tokenizer(text, return_tensors="pt")
print(inputs)


input_ids — числовые представления слов, attention_mask указывает, какие токены учитывать при обработке.

## 4. Работа с моделью: Анализ тональности
Теперь передадим данные в модель:



In [ ]:
from torch.nn.functional import softmax

outputs = model(**inputs)
probs = softmax(outputs.logits, dim=1)

print(f"Положительная тональность: {probs[0][1].item():.4f}")
print(f"Отрицательная тональность: {probs[0][0].item():.4f}")


Результат: модель выдаст вероятности тональности (положительная или отрицательная).

## 5. Использование конвейеров (pipelines)
В transformers есть удобный инструмент pipeline, который автоматически подбирает нужную модель и обрабатывает текст без сложной настройки. Поддерживаются разные задачи, такие как анализ тональности, генерация текста, ответы на вопросы и др. Попробуем использовать конвейеры для анализа тональности, генерации текста и ответов на вопросы.
Название конвейера указывается в качестве аргумента pipeline. Например, для анализа тональности используется "sentiment-analysis". Для генерации текста — "text-generation". Для ответов на вопросы — "question-answering". Для сжатия текста — "summarization". Для перевода — "translation_xx_to_yy" (где xx и yy — языки перевода).

✅ Пример анализа тональности:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
print(classifier("Hugging Face is amazing!"))


## Пример генерации текста с GPT-2:

In [ ]:
text_generator = pipeline("text-generation", model="gpt2")
print(text_generator("Once upon a time", max_length=50))


## Пример ответа на вопросы:

In [ ]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

context = "Hugging Face is a company that specializes in Natural Language Processing."
question = "What does Hugging Face specialize in?"

result = qa_pipeline(question=question, context=context)
print(result)


## 6. Как найти и использовать другие бесплатные модели?
Hugging Face Hub (https://huggingface.co/models) содержит тысячи бесплатных моделей. Можно искать по ключевым словам и загружать нужную модель так:


    model_name = "facebook/bart-large-mnli"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

### Пример нулевого (zero-shot) классификатора:

Такой классификатор, может предсказать, что образец соответствует некоторому классу, даже если такие образцы не наблюдались во время обучения

    zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    
    sequence = "I want to book a flight to Paris."
    labels = ["travel", "finance", "sports"]
    
    print(zero_shot_classifier(sequence, candidate_labels=labels))




In [ ]:
model_name = "facebook/bart-large-mnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:


zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

sequence = "I want to book a flight to Paris."
labels = ["travel", "finance", "sports"]

print(zero_shot_classifier(sequence, candidate_labels=labels))

### ЗАДАНИЕ
1. Выберите другую модель из Hugging Face Hub и загрузите её. Модель должна поддерживать русский язык
2. Выполните анализ тональности текста с использованием новой модели.
В качестве примеров используйте отзывы на фильмы или продукты на русском языке.
3. Выберите модель для ответов на вопросы и выполните задание 6 для русского языка.
4. Найдите модель для русского языка и выполните суммаризацию (сжатие, краткий пересказ) текста первых трех глав Гарри Поттера.

    summarizer = pipeline("summarization", model="...")
5. Найдите датасет с песнями на русском языке. выполните парафраз текстов песен. _pipeline("text2text-generation" ....)_



In [1]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    res = {0: 'NEUTRAL', 1: 'POSITIVE', 2: 'NEGATIVE'}
    result = [res[i] for i in predicted]
    return result

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [2]:
predict(['Еда норм. Не понравилось, что долго нужно стоять в очереди, потому что все киоски для заказа закрыты и не работают',
         '12312', 'Кофе как вода.', 'Всегда очень много народу и прекрасный выбор за минимальный бюджет . Можно перекусить в кафе а можно и взять с собой. Все аккуратно упакуют',
         'Данный фильм могут смотреть наркоманы солевики, игра актёров такая что в СССР за такое расстреляли на съёмочной площадке. Однако в титрах столько людей что кажется там должно было быть месиво как во второй мировой. Графика норм но без идеи и без какой либо мотивации игры актёров данный отрывок я считаю что это даже не фильм ',
         'Фильм Феликса Умарова — это не классический байопик, а дерзкий микс мюзикла, драмы и рэп-баттлов. Сюжет охватывает ключевые этапы жизни Пушкина: от лицейских бунтов до дуэли с Дантесом. Однако вместо исторической скрупулёзности зритель получает ',
         ''])

['NEGATIVE', 'NEUTRAL', 'NEUTRAL', 'POSITIVE', 'NEUTRAL', 'NEUTRAL', 'NEUTRAL']

In [ ]:
from huggingface_hub import login
login(token="hf_token")

In [ ]:
# https://huggingface.co/t-bank-ai/ruDialoGPT-small

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-small')
inputs = tokenizer('@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@', return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=10,
    top_p=0.95,
    num_beams=3,
    num_return_sequences=3,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.2,
    repetition_penalty=1.2,
    length_penalty=1.0,
    eos_token_id=50257,
    max_new_tokens=40
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response


special_tokens_map.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:1905: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@нормально, я не курю и не употребляю алкоголь, а ты? :D привет)@@ВТОРОЙ@@привет) я тоже не пью, но у меня есть девушка, которая не курит и',
 '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@нормально, я не курю и не употребляю алкоголь, а ты? :D привет)@@ВТОРОЙ@@привет) я тоже не пью, но у меня есть девушка, которая не курит,',
 '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@нормально, я не курю и не употребляю алкоголь, а ты? :D привет)@@ВТОРОЙ@@привет) я тоже не пью, но у меня есть девушка, которая не курит.']

In [18]:
def pred(text):
    inputs = tokenizer(text, return_tensors='pt')
    generated_token_ids = model.generate(
      **inputs,
      top_k=10,
      top_p=0.95,
      num_beams=3,
      num_return_sequences=3,
      do_sample=True,
      no_repeat_ngram_size=2,
      temperature=1.2,
      repetition_penalty=1.2,
      length_penalty=1.0,
      eos_token_id=50257,
      max_new_tokens=40
    )
    context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
    print(context_with_response)

In [19]:
pred('Кто написал пьесу «Ромео и Джульетта»?')

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['Кто написал пьесу «Ромео и Джульетта»?@@ВТОРОЙ@@Сценарий писал Пушкин, а не Пушкин.@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@', 'Кто написал пьесу «Ромео и Джульетта»?@@ВТОРОЙ@@Сценарий писал Пушкин, а не Дюма.@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@', 'Кто написал пьесу «Ромео и Джульетта»?@@ВТОРОЙ@@Сценарий писал Пушкин, а пьесу написал Гоголь.@@ПЕРВЫЙ@@']


In [20]:
pred('Сколько континентов на Земле?')

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['Сколько континентов на Земле?@@ВТОРОЙ@@Смотря что считать континентом.@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@', 'Сколько континентов на Земле?@@ВТОРОЙ@@Смотря что считать континентом. А то и весь материк.@@ПЕРВЫЙ@@', 'Сколько континентов на Земле?@@ВТОРОЙ@@Смотря что считать континентом. А то и весь мир.@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@']


In [21]:
pred('Какая самая длинная река в мире?')

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


['Какая самая длинная река в мире?@@ВТОРОЙ@@Америка. Самая большая река на Земле.@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@', 'Какая самая длинная река в мире?@@ВТОРОЙ@@Америка. Самая большая река на планете.@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@@@ПЕРВЫЙ@@', 'Какая самая длинная река в мире?@@ВТОРОЙ@@Америка. Самая большая река на планете. И самая глубокая.@@ПЕРВЫЙ@@']


In [ ]:
# https://huggingface.co/LaciaStudio/Lacia_sum_small_v1

In [1]:
# Summarization
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("LaciaStudio/Lacia_sum_small_v1")
model = AutoModelForSeq2SeqLM.from_pretrained("LaciaStudio/Lacia_sum_small_v1")

text = "Современные технологии оказывают значительное влияние на нашу повседневную жизнь и рабочие процессы. Искусственный интеллект становится важным инструментом, помогающим оптимизировать задачи и открывающим новые перспективы в различных областях."

# "summarize: " prefix
input_text = "summarize: " + text
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:", summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/21.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

Summary: Современные технологии оказывают значительное влияние на повседневную жизнь и рабочие процессы, включая искусственный интеллект, который помогает оптимизировать задачи и открывать новые перспективы.


In [16]:
def summ(text):
  input_text = "summarize: " + text
  inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

  summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  # print("Summary:", summary)
  return summary

In [3]:
with open('HP1RUS.TXT', encoding='windows-1251') as f:
       # print(f.read().split('\n')[892:984])
       print(f.read().split('\n')[4:7])

['   Глава первая ', '   Мальчик, который выжил ', '    ']


In [4]:
text = ''

with open('HP1RUS.TXT', encoding='windows-1251') as f:
   text = '\n'.join(f.read().split('\n')[4:984])
len(text)

63889

In [6]:
summ(text)

Summary: Мистер и миссис Дарсли, выжившие в доме номер четыре по Бирючинному проезду, могли бы стать последними, если бы они не участвовали в чем-то странном и таинственном. Они были директором фирмы Граннингз, которая выпускала сверла, и их дочь, дочь Дадли, боялась, что кто-нибудь раскроет их тайну. У них также был маленький сын, который не видел их с тех пор, как они увидели Поттеров. В серый пасмурный вторник они столкнулись с необычными и таинственными событиями, в результате чего начались странные и странные события, такие как утрата галстука, а в половине девятого они целовали друг друга, но безуспешно.


In [19]:
sum_ = ''
for i in range(len(text)//1000 + 1):
  # print(len(text[i*1000:(i+1)*1000]))
  part = text[i*1000:(i+1)*1000]
  sum_ += str(summ(part))
  print(i, end=' ')

sum_

0 1 2 3 

KeyboardInterrupt: 

In [20]:
sum_

'Мистер и миссис Дарсли, выжившие в доме номер четыре по Бирючинному проезду, могли бы стать последними, от кого бы они могли ожидать участия в чем-то странном и таинственном. Они работали директором фирмы Граннингз, выпускающей сверла, и были худыми и светловолосыми, а их шеи с лихвой хватило бы на двоих. У них рос сын Дадли, который боялся, что кто-нибудь раскроет их тайну.Миссис Поттер приходилась сестрой, которая не видела ее уже несколько лет. Миссис Дарсли делала вид, что у нее вообще нет сестры, потому что ее никчемный муж и ее муж были не похожи на нее. Она ожидала, когда соседи увидят ее, так как у них тоже есть маленький сын. Однако они не хотели, чтобы Дадли общался с подобными детьми. В серый пасмурный вторник начался странный и таинственный процесс, в результате чего начались странные события, включая мурлычок и водружение на высокий стул. Никто не заметил золотисто-коричневую сову.Мистер Дарсли взял чемоданчик, чмокнул миссис Дарсел и попытался поцеловать Дадли на прощани

In [1]:
# Найдите датасет с песнями на русском языке.
# выполните парафраз текстов песен. pipeline("text2text-generation" ....)

In [4]:
# !pip install datasets

In [ ]:
# https://huggingface.co/openai/whisper-large-v2

In [5]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.config.forced_decoder_ids = None

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [6]:
transcription

[' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.']

In [ ]:
# https://huggingface.co/datasets/bond005/sberdevices_golos_100h_farfield

In [7]:
dataset = load_dataset("bond005/sberdevices_golos_100h_farfield")

README.md:   0%|          | 0.00/6.67k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

(…)-00000-of-00003-066abd799f2fa68d.parquet:   0%|          | 0.00/286M [00:00<?, ?B/s]

(…)-00001-of-00003-5a54f288ebe5fa98.parquet:   0%|          | 0.00/288M [00:00<?, ?B/s]

(…)-00002-of-00003-d00e7bb216665874.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

(…)-00000-of-00001-fd7b65d8512dcdb5.parquet:   0%|          | 0.00/85.6M [00:00<?, ?B/s]

(…)-00000-of-00001-7fb58d4cda8ff9fe.parquet:   0%|          | 0.00/130M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9570 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/933 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1916 [00:00<?, ? examples/s]

In [21]:
audio = dataset['test']['audio'][0]['array']
audio, dataset['test']['transcription'][0]

(array([-3.05175781e-05, -3.05175781e-05,  0.00000000e+00, ...,
         3.05175781e-05,  3.05175781e-05,  0.00000000e+00]),
 'салют сколько предприятий являются клиентами сбера')

In [15]:
dataset, sample

(DatasetDict({
     train: Dataset({
         features: ['audio', 'transcription'],
         num_rows: 9570
     })
     validation: Dataset({
         features: ['audio', 'transcription'],
         num_rows: 933
     })
     test: Dataset({
         features: ['audio', 'transcription'],
         num_rows: 1916
     })
 }),
 {'path': '1272-128104-0000.flac',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
         0.0010376 ]),
  'sampling_rate': 16000})

In [28]:
input_features = processor(audio, return_tensors="pt").input_features
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[' Салют! Сколько предприятий являются клиентами Сбера?']

In [35]:
from datasets import Dataset, Audio
audio_dataset = Dataset.from_dict({"audio": ['song1.mp3', 'song2.mp3']}).cast_column("audio", Audio())

In [32]:
audio_dataset['audio'][0]

{'path': 'song1.mp3',
 'array': array([0., 0., 0., ..., 0., 0., 0.]),
 'sampling_rate': 44100}

In [37]:
input_features = processor(audio_dataset['audio'][1]['array'], return_tensors="pt").input_features # , sampling_rate=audio_dataset['audio'][0]['sampling_rate']
forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[' Пусть будет лето']

# Что такое datasets в Hugging Face?
Hugging Face предоставляет библиотеку datasets, которая содержит тысячи бесплатных наборов данных для NLP и других задач машинного обучения.

✅ Основные преимущества datasets:
✔ Поддержка больших наборов данных (работает с памятью эффективно)
✔ Можно загружать, обрабатывать и тренировать модели
✔ Поддержка многих форматов данных (CSV, JSON, Parquet и т. д.)
✔ Готовые стандартные датасеты для NLP (SQuAD, IMDB, WikiText и др.)

# Пример загрузки набора данных
Загрузим популярный IMDb (анализ тональности):

### Что мы увидим?

Датасет состоит из тренировочного (train) и тестового (test) наборов
В каждом 25 000 примеров
Колонки text и label
text – отзыв
label – метка (0 – негативный, 1 – позитивный)

In [ ]:
from datasets import load_dataset

# Загружаем датасет IMDb
dataset = load_dataset("imdb")

# Смотрим структуру данных
print(dataset)


# Доступ к данным
Посмотрим первые примеры из датасета:

In [ ]:
print(dataset["train"][0])  # Вывод первого примера


# Что такое features в Hugging Face?
В Hugging Face features – это признаки (колонки) набора данных.

💡 Как посмотреть features (структуру данных)?

In [ ]:
print(dataset["train"].features)


# Что означают features
# text: строка (сам отзыв)
# label: классификационная метка (0 – "negative", 1 – "positive")
